# Import Libraries


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import datetime
import warnings
warnings.filterwarnings("ignore")

# Import Datasets

In [2]:
path_to_orders_data='./input_data/machine_learning_challenge_order_data.csv'
path_to_labelled_data='./input_data/machine_learning_challenge_labeled_data.csv'


In [3]:
#Import orders data and display few rows
df_orders=pd.read_csv(path_to_orders_data)
df_orders.head()

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,000097eabfd9,2015-06-20,19,1.0,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,0000e2c6d9be,2016-01-29,20,1.0,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,000133bb597f,2017-02-26,19,1.0,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324
3,00018269939b,2017-02-05,17,1.0,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356
4,0001a00468a6,2015-08-04,19,1.0,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356


In [5]:
#Import labelled data and display few rows
df_target_label=pd.read_csv(path_to_labelled_data)
df_target_label.head()

,customer_id,is_returning_customer
0,000097eabfd9,0
1,0000e2c6d9be,0
2,000133bb597f,1
3,00018269939b,0
4,0001a00468a6,0


In [6]:
df_orders.shape

(786600, 13)

In [8]:
df_target_label.shape

(245455, 2)

In [13]:

df_orders.customer_id.nunique()

245455

We have 786600 records of orders for 245,455 unique customers in  2yrs span of time (2015-03-01,2017-02-28) 

In [11]:
##Checking for any duplicate values. If any will be removed accordingly.
df_orders=df_orders.drop_duplicates()
df_orders.shape

(786054, 13)

In [12]:
#Checking out  missing values if any in the columns of order data.
df_orders.isnull().any()

customer_id            False
order_date             False
order_hour             False
customer_order_rank     True
is_failed              False
voucher_amount         False
delivery_fee           False
amount_paid            False
restaurant_id          False
city_id                False
payment_id             False
platform_id            False
transmission_id        False
dtype: bool

As mentioned in data dictionary that 'customer_order_rank' is empty for failed orders.

In [14]:
#Datatypes of columns of the orders dataframe
df_orders.dtypes

customer_id             object
order_date              object
order_hour               int64
customer_order_rank    float64
is_failed                int64
voucher_amount         float64
delivery_fee           float64
amount_paid            float64
restaurant_id            int64
city_id                  int64
payment_id               int64
platform_id              int64
transmission_id          int64
dtype: object

'order_date' need to be converted to datetime format as its one of the imporant column to understand the order patterns with respect to days etc. rest all columns seems to be fine

In [15]:
##Converting into Timestamp(datetime format)
df_orders['order_date']=pd.to_datetime(df_orders['order_date'])
df_orders.head()

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,000097eabfd9,2015-06-20,19,1.0,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,0000e2c6d9be,2016-01-29,20,1.0,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,000133bb597f,2017-02-26,19,1.0,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324
3,00018269939b,2017-02-05,17,1.0,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356
4,0001a00468a6,2015-08-04,19,1.0,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356


Describing the data to see count,min,max values of the various numerical columns 

In [18]:
df_orders[['customer_order_rank','voucher_amount','delivery_fee','amount_paid']].describe()

,customer_order_rank,voucher_amount,delivery_fee,amount_paid
count,761833.000000,786054.000000,786054.000000,786054.000000
mean,9.436810,0.091476,0.181161,10.182875
std,17.772322,0.479488,0.369661,5.605253
min,1.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,6.653430
50%,3.000000,0.000000,0.000000,9.027000
75%,10.000000,0.000000,0.000000,12.213000
max,369.000000,93.398900,9.860000,1131.030000


In [19]:
df_orders[df_orders.customer_order_rank==369]

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
69538,15edce943edd,2017-02-26,19,369.0,0,0.0,0.0,7.5402,332363498,31506,1779,29463,4996


# keytakeaways:
customer_order_rank has missing values and with average of 9-10 orders and also max number of orders being 369 by one of the customer_id(15edce943edd).

voucher_amount seems to be 0 for most of them and ofcourse max value for one of the customer being 93.39.

delivery_fee seems to be low to 0 for most of them and on average being 0.18 collected.

amount_paid per order is in range of 0 to 1131 .its been  10 on average though

In [20]:
##Categorical variables distribution
cat_columns = ['restaurant_id','city_id','payment_id','transmission_id','platform_id','order_hour','is_failed']
df_orders[cat_columns] = df_orders[cat_columns].astype(str)
df_orders.describe(include='O')

,customer_id,order_hour,is_failed,restaurant_id,city_id,payment_id,platform_id,transmission_id
count,786054,786054,786054,786054,786054,786054,786054,786054
unique,245455,24,2,13569,3749,5,14,10
top,15edce943edd,19,0,37623498,10346,1619,29463,4356
freq,386,133959,761833,1314,86574,476507,241351,341675


top:Most commonly occuring value among all values in a column.
freq:Frequency of most commonly occuring value amoong all values in a column

# Keytakeaways:
orders data is from 3,749 unique cities,14 unique platforms,5 unique payment methods,10 different transmissions.

Most commonly order_hour seems to 19 hrs(but depends on timezone of data)

Failed orders represented by is_failed=1 and seems to be less amount of data. As is_failed=0 has more values.


In [22]:
# Labelled data understanding
#Count of customers by target response variable 'is_returning_customer'
df_target_label.is_returning_customer.value_counts()

0    189948
1     55507
Name: is_returning_customer, dtype: int64

In [24]:
#Percentage of customers
df_target_label.is_returning_customer.value_counts(normalize=True)*100

0    77.386079
1    22.613921
Name: is_returning_customer, dtype: float64

# Keytakeaways

we have almost 77.3% of customers who churn out and dont return to the platform(did not order again in the 6 months after 2017-02-28).

22.6% of customers are placing orders again at least once after 2017-02-28.                                                                     
Hence it becomes a class imbalance problem which needs to be balanced during train and test split of datasets and further evaluations.

                                                                           

# Feature Engineering and Transformations

Lets transform the data into rows per customer aggreagted information by computing various features through feature engineering.


In [26]:
#Input dataset
df_orders.head()

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
0,000097eabfd9,2015-06-20,19,1.0,0,0.0,0.000,11.46960,5803498,20326,1779,30231,4356
1,0000e2c6d9be,2016-01-29,20,1.0,0,0.0,0.000,9.55800,239303498,76547,1619,30359,4356
2,000133bb597f,2017-02-26,19,1.0,0,0.0,0.493,5.93658,206463498,33833,1619,30359,4324
3,00018269939b,2017-02-05,17,1.0,0,0.0,0.493,9.82350,36613498,99315,1619,30359,4356
4,0001a00468a6,2015-08-04,19,1.0,0,0.0,0.493,5.15070,225853498,16456,1619,29463,4356


Compute  
total orders placed per customer and also count of successful and failed orders 

In [27]:
#total orders
df_count_orders=df_orders[['customer_id','order_date']].groupby('customer_id')['order_date'].count().reset_index(name='total_orders')
df_count_orders.head()


,customer_id,total_orders
0,000097eabfd9,1
1,0000e2c6d9be,1
2,000133bb597f,1
3,00018269939b,1
4,0001a00468a6,1


In [29]:
#total successful orders ie filtering orders with customer_order_rank is null (failed orders)
df_count_sucess_orders=df_orders[~df_orders.customer_order_rank.isnull()][['customer_id','customer_order_rank']]
df_count_sucess_orders=df_count_sucess_orders.groupby('customer_id')['customer_order_rank'].count().reset_index(name='total_sucess_orders')
df_count_sucess_orders.head()


,customer_id,total_sucess_orders
0,000097eabfd9,1
1,0000e2c6d9be,1
2,000133bb597f,1
3,00018269939b,1
4,0001a00468a6,1


In [33]:
#Total failed orders per customer
df_orders['is_failed']=df_orders['is_failed'].astype(int)
df_count_failed_orders=df_orders[['customer_id','is_failed']].groupby('customer_id')['is_failed'].sum().reset_index(name='total_failed_orders')
df_count_failed_orders.head()


,customer_id,total_failed_orders
0,000097eabfd9,0
1,0000e2c6d9be,0
2,000133bb597f,0
3,00018269939b,0
4,0001a00468a6,0


In [39]:
#Just cross checking the computation for one random customer with orders information as well
df_count_failed_orders[df_count_failed_orders.customer_id=='fffe9d5a8d41'].head()

,customer_id,total_failed_orders
245451,fffe9d5a8d41,2


In [37]:
df_orders[df_orders.customer_id=='fffe9d5a8d41'].head()

,customer_id,order_date,order_hour,customer_order_rank,is_failed,voucher_amount,delivery_fee,amount_paid,restaurant_id,city_id,payment_id,platform_id,transmission_id
786593,fffe9d5a8d41,2016-07-31,21,NaN,1,0.0,0.0,8.4429,156133498,10346,1811,29463,212
786594,fffe9d5a8d41,2016-09-30,20,1.0,0,0.0,0.0,10.7262,983498,10346,1779,29463,4228
786595,fffe9d5a8d41,2016-09-30,20,NaN,1,0.0,0.0,10.7262,983498,10346,1779,29463,212


In [30]:
df_count_sucess_orders.customer_id.nunique()

244937

# Keytakeaways

we have 244937 customers of 245455 ie almost 99.7% customers having placed atleast one successful order.
Remaining 518 customers have placed orders but havent been successful maybe platform issues/restaurant decline or change of user choice etc could be reasons/ data capturing problems

For further analysis because of time constraints and also 518 customers seems to be of small magnitude .I would consider the data as it and won't filter those failed orders customers from the dataset.
In future areas of imporvement ,this would need further investigation to filter and build datasets separately to see if it has impact on model performances etc